In [ ]:
import numpy as np
import torch
import sys, os

sys.path.append("../")
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter, NullFormatter

seed = 1
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

%matplotlib inline

### Loading data

We will use the `pytorch` `Dataset` class for handling the data.

Here we wil load some data of 40 artificial neurons that implement a noisy oscillator. This data is an array of dimensions `num_units x sequence_length`, which we use to initialise a `Basic_dataset`.

If one has data with trial structure one can instead initialise the `Basic_dataset_with_trials` class using an array of dimensions `num_trials x num_units x sequence_length`.

In [ ]:
from vi_rnn.datasets import Basic_dataset

In [ ]:
# load data
data_all = np.load("tutorial_data/spiking_data.npy")

# split into train and eval
eval_split_ts = int(.75*data_all.shape[1])
data_train = data_all[:,:eval_split_ts]
data_eval = data_all[:,eval_split_ts:]


# initialise a dataset class
task_params = {"name": "tutorial_cont",
               "dur": 100, # we will sample pseudo trials of "dur" timesteps during training
               "n_trials": 400 # every epoch consists of 256 psuedo trials
               }
dataset = Basic_dataset(
    task_params=task_params,
    data=data_train, 
    data_eval=data_eval, 
    stim = None, # you could additionally pass input / stimuli like this
    stim_eval = None,  
)

In [ ]:
# plot some example data
fig, ax = plt.subplots(1, 1, figsize=(6, 2))
ax.imshow(dataset.data[:,:1000],aspect='auto',cmap="Grays",vmax=2)
ax.set_xlabel("timesteps")
ax.set_ylabel("neurons")

### Initialise the variational inference model

All the inference functions, and the model parameters are contained in the `VAE` class, which contains 

- All the RNN parameters in: `vae.rnn`
    - The dynamics / transition model $p(\mathbf{z}_t\mid\mathbf{z}_{t-1})$ is in: `vae.rnn.transition`
    - The observation model $p(\mathbf{y}_t\mid\mathbf{z}_{t})$ is in: `vae.rnn.observation`
- If used, the encoder  $e(\mathbf{z}_t\mid\mathbf{y}_{t:t+t'})$ parameters are in `vae.encoder`. Here we will use a CNN encoder (with temporal convolutions and effective window size $t'$)

The proposal distribution $r$ from which we sample latents is defined as a combination of the RNN dynamics and the encoder's prediction

$$r(\mathbf{z}_t\mid\mathbf{z}_{t-1},\mathbf{y}_{t:t+t'})\propto e(\mathbf{z}_t\mid\mathbf{y}_{t:t+t'})p(\mathbf{z}_t\mid\mathbf{z}_{t-1})$$

We will here use a one-to-one mapping between RNN units and observations. Note that generally one does not have to do this - if you want to learn a linear map between RNN units / latents and observations set `rnn_params["identity_readout"]` to `False`.


In [ ]:
from vi_rnn.vae import VAE

In [ ]:
enc_params = (
    {"kernel_sizes": [21, 11, 1], # kernel sizes of the CNN
    "padding_mode": "constant", # padding mode of the CNN (e.g., "circular", "constant", "reflect")
    "nonlinearity": "gelu", # "leaky_relu" or "gelu"
    "n_channels": [64,64], # number of channels in the CNN (last one will be equal to dim_z)
    "init_scale": 0.1, # initial scale of the noise predicted by the encoder
    "constant_var": False, # whether or not to use a constant variance (as opposed to a data-dependent variance)
    "padding_location": "acausal",} # padding location of the CNN ("causal", "acausal", or "windowed")
)  
rnn_params = {
    # noise covariances settings
    "train_noise_z": True,  # whether or not to train the transition noise scale
    "train_noise_z_t0": True,  # whether or not to train the initial state noise scale
    "init_noise_z": 0.1,  # initial scale of the transition noise
    "init_noise_z_t0": 0.1,  # initial scale of the initial state noise
    "noise_z": "diag",  # transition noise covariance type ("full", "diag" or "scalar"), set to "full" when using the optimal proposal
    "noise_z_t0": "diag",  # initial state noise covariance type ("full", "diag" or "scalar"), set to "full" when using the optimal proposal
    "transition": "low_rank",
    # readout settings
    "observation": "one_to_one",  # if True enforces a one to one mapping between RNN units and recorded units
    "readout_from": "currents",  # set to "currents", "rates", "z" or "z_and_v". We can readout from the RNN activity
    # before / after applying the non-linearty by setting this to "currents" / "rates" respectively.
    # Alternatively we can directly readout from the latent dynamics z of the RNN by
    # setting this to "z", or from latents z and input v, by setting this to "z_and_v"
    "train_obs_bias": True,  # whether or not to train a bias term in the observation model
    "train_obs_weights": True,  # whether or not train the weights of the observation model
    "obs_nonlinearity": "softplus",  # can be used to rectify the output (e.g., when using Poisson observations, use "softplus")
    "obs_likelihood" : "Poisson",

    # other
    "activation": "relu",  # set the nonlinearity to "clipped_relu, "relu", "tanh" or "identity"
    "decay": 0.9,  # initial decay constant, scalar between 0 and 1
    "train_neuron_bias": True,  # train a bias term for every neuron
    "weight_dist": "uniform",  # weight distribution ("uniform" or "gauss")
    "initial_state": "trainable",  # initial state ("trainable", "zero", or "bias")
    "simulate_input": False
}


VAE_params = {
    "dim_x": 40,  # observation dimension (number of units in the data)
    "dim_z": 2,  # latent dimension / rank of the RNN
    "dim_N": 40,  # amount of units in the RNN (can generally be different then the observation dim)
    "dim_u": 0,  # input stimulus dimension
    "enc_architecture": "CNN",  # encoder architecture (not trained when using linear Gauss observations)
    "enc_params": enc_params,  # encoder paramaters
    "rnn_architecture": "LRRNN",  # use a low-rank RNN architecture
    "rnn_params": rnn_params,  # parameters of the RNN
}

# initialise the VAE
vae = VAE(VAE_params)

### Training

We will now fit an RNN to the data, using the `train_VAE` function. There are two inference functions that can be used during training:
- `training_params["loss_f"] = "opt_smc"` uses `vae.forward_optimal_proposal()`. This can be used with linear observations and inverts the observation model closed-form.
- `training_params["loss_f"] = "smc"` uses `vae.forward()`. This can be used for arbitrary observation models, and uses an encoder to update the proposal distribution.

Here we use the second as observations model is non-linear.

In [ ]:
from vi_rnn.saving import save_model, load_model
from vi_rnn.train import train_VAE

In [ ]:
training_params = {
    "lr": 1e-3,  # learning rate start
    "lr_end": 1e-5,  # learning rate end (with exponential decay)
    "n_epochs": 1000,  # number of epochs to train
    "grad_norm": 0,  # gradient clipping above certain norm (if this is set to >0)
    "batch_size": 10,  # batch size
    "cuda": False,  # train on GPU
    "k": 64,  # number of particles to use
    "loss_f": "smc",  # use regular variational SMC ("smc"), or use the optimal ("opt_smc")
    "resample": "systematic",  # type of resampling "systematic", "multinomial" or "none"
    "run_eval": False,  # run an evaluation setup during training (requires additional parameters)
    "observation_likelihood": "Poisson",  # "Gauss"-ian  or "Poisson" observations
    "t_forward":0, # timesteps to predict without using the encoder
    "sim_v": False,  # set to True when using time-varying inputs
}

In [ ]:
# run training

train_VAE(
    vae,
    training_params,
    dataset,
    sync_wandb=False,
    out_dir="tutorial_data",
    fname="tutorial_spikes_new",
)

In [ ]:
# As we only trained for a couple of epochs we will load a pretrained model
# This was trained as above, but for 500 epochs
save_model(
    vae, training_params, task_params, "tutorial_data/test"
)

In [ ]:
vae, training_params, task_params = load_model(
    "tutorial_data/test"
)

In [ ]:
# As we only trained for a couple of epochs we will load a pretrained model
# This was trained as above, but for 2000 epochs
#fname ="tutorial_data/tutorial_spikes"
#vae, training_params, task_params  = load_model(
#   fname, backward_compat=True
#)

### Plot the trained model's output

To generate data from our trained model, we can use the `predict` function. We can either use a data-inferred initial state, by passing the data and the argument `initial_state="posterior_mean"` or just sample from the RNNs initial state by passing `initial_state="prior_mean"`.

Generally we get nicer visualisation of the latent dynamics by projecting the data on an orthogonalised basis. We here use the basis spanned by the left singular vectors of the RNNs weight matrix, which we obtain using the function `get_orth_proj_latents`.

In [ ]:
from vi_rnn.predict import predict
from vi_rnn.utils import get_orth_proj_latents, orthogonalise_network
from evaluation.calc_stats import calc_isi_stats,calculate_correlation

In [ ]:
# generate data using our fit model
Z_gen, data_gen, rates_gen = predict(
    vae,
    u=None, # add optional input / stimuli
    x=dataset.data_eval,
    initial_state="posterior_mean",
)

In [ ]:
vae = orthogonalise_network(vae)
vae.rnn.std_embed_z(vae.rnn.R_z)*torch.mean(vae.rnn.observation.B)/4


In [ ]:
dy = 10
dx = 5
B = torch.zeros(dy, dx)
B[range(dx), range(dx)] = 1
B = torch.nn.Parameter(B, requires_grad=True)
mask = B
cast_B = lambda x: x * mask

In [ ]:
# orthogonalise the latents
projection_matrix = get_orth_proj_latents(vae)
Z_gen_orth = np.einsum("BZT,OZ->BOT", Z_gen.squeeze(-1), projection_matrix)

In [ ]:
# plot data generated by our model next to evaluation data
# NOTE: we do not expect a timestep wise correspondance, as we are plotting samples from a stochastic
# model. Instead we expect the distributions over samples to match.

fig, ax = plt.subplots(3)

ax[0].set_ylabel("Observed activity")
ax[0].imshow(dataset.data_eval[:,:1000],aspect='auto',cmap="Grays",vmax=2)

ax[1].set_ylabel("Generated activity")
ax[1].imshow(data_gen[0],aspect='auto',cmap="Grays",vmax=2)


ax[2].set_ylabel("Generated latents")
ax[2].plot(Z_gen_orth[0].T)
ax[2].set_xlabel("timesteps")

for a in ax.flatten():
    a.set_xlim(0, 1000)

In [ ]:
# Calculate some statistics to compare the true and generated data

spikes_GT = dataset.data_eval.T.detach().numpy()
spikes_gen = data_gen[0,:,:,0].T.detach().numpy()

fig, ax = plt.subplots(1, 3, figsize=(3, 1))

# CV and ISI
bins = np.linspace(0, 100, 100)
CVs_isi_gen, Means_isi_gen, Std_isi_gen = calc_isi_stats(spikes_gen)
CVs_isi_GT, Means_isi_GT, Std_isi_GT = calc_isi_stats(spikes_GT)
Means_gen = np.mean(spikes_gen, axis=0)
Means_GT = np.mean(spikes_GT, axis=0)


ax[0].set_ylabel("Generated")
ax[0].set_xlabel("Data")

ax[1].scatter(Means_isi_GT, Means_isi_gen, color="teal", alpha=0.7, s=20, linewidth=0)
ax[1].set_title("mean ISIs")
max = np.max([Means_isi_GT, Means_isi_gen]) * 1.1
ax[1].plot([0, max], [0, max], color="grey", linestyle="--", zorder=-1000)
ax[1].set_aspect(1)

ax[0].scatter(Means_GT, Means_gen, color="teal", alpha=0.7, s=20, linewidth=0)
ax[0].set_title("mean rates")
max = np.max([Means_gen, Means_GT]) * 1.1
ax[0].plot([0, max], [0, max], color="grey", linestyle="--", zorder=-1000)
ax[0].set_aspect(1)

ax[0].set_yscale("log")
ax[0].set_xscale("log")
ax[0].set_xticks([0.1, 0.3])
ax[0].set_yticks([0.1, 0.3])
ax[1].set_xlim(0)
ax[1].set_ylim(0)

# format axis
ax[0].yaxis.set_major_formatter(StrMethodFormatter("{x:.1f}"))
ax[0].yaxis.set_minor_formatter(NullFormatter())
ax[0].xaxis.set_major_formatter(StrMethodFormatter("{x:.1f}"))
ax[0].xaxis.set_minor_formatter(NullFormatter())
ax[0].set_yticklabels([])
ax[1].set_yticklabels([])

# Plot pairwise correlations
correlation_GT = calculate_correlation(spikes_GT)
correlation_gen = calculate_correlation(spikes_gen)

i_upper = np.triu_indices(40, k=1)
corr_values_GT = correlation_GT[i_upper]
corr_values_gen = correlation_gen[i_upper]

ax[2].scatter(
   corr_values_GT, corr_values_gen, color="teal", alpha=0.2, linewidth=0, s=10
)
ax[2].set_aspect(1)

ax[2].set_title("pairwise correlations")
ax[2].set_yticklabels([])
max = np.max([corr_values_GT, corr_values_gen]) * 1.1
ax[2].plot([0, max], [0, max], color="grey", linestyle="--", zorder=-10)
